#### [bosch kaggle info](https://www.kaggle.com/c/bosch-production-line-performance/data)

[pyspark doc](https://spark.apache.org/docs/2.0.0-preview/api/python/_modules/pyspark/sql/session.html)

** file path **
/home/h/fun/bosch/raw_data

Data Files

- ** 目标是要预测哪个部分会在质量控制过程中失败，用'Response' = 1表示该状态。**
- ** 数据集包含了非常大的匿名特征数据。**
- ** 数据已经按照类型切成两类，数值型和类别型两个大文件。日期特征给每个测量值提供了发生的时间戳。每个日期列都对应于每个特征的生成日期，如L0_S0_D1是特征列L0_S0_F0的日期列。 ** 
- ** 难点： 量大，正负样本的极度不均衡 **

File descriptions

* train_numeric.csv - the training set numeric features (this file contains the 'Response' variable) 
* test_numeric.csv - the test set numeric features (you must predict the 'Response' for these Ids)  
* train_categorical.csv - the training set categorical features  
* test_categorical.csv - the test set categorical features  
* train_date.csv - the training set date features  
* test_date.csv - the test set date features  
* sample_submission.csv - a sample submission file in the correct format 

In [35]:
%cd raw_data/
%ls

/home/h/kaggle_fun/bosch/raw_data
down_data/            test_numeric.csv       train_numeric.csv
test_categorical.csv  train_categorical.csv  zips/
test_date.csv         train_date.csv


In [15]:
import pandas as pd
import os

In [16]:
train_date = pd.read_csv("raw_data/train_date.csv", nrows=100)
train_category = pd.read_csv("raw_data/train_categorical.csv", nrows=100)
train_numer = pd.read_csv("raw_data/train_numeric.csv", nrows=1000)

In [17]:
train_category.head()

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
train_date.head()

,Id,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,...,L3_S50_D4246,L3_S50_D4248,L3_S50_D4250,L3_S50_D4252,L3_S50_D4254,L3_S51_D4255,L3_S51_D4257,L3_S51_D4259,L3_S51_D4261,L3_S51_D4263
0,4,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
train_numer.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [20]:
train_numer.describe

<bound method DataFrame.describe of        Id  L0_S0_F0  L0_S0_F2  L0_S0_F4  L0_S0_F6  L0_S0_F8  L0_S0_F10  \
0       4     0.030    -0.034    -0.197    -0.179     0.118      0.116   
1       6       NaN       NaN       NaN       NaN       NaN        NaN   
2       7     0.088     0.086     0.003    -0.052     0.161      0.025   
3       9    -0.036    -0.064     0.294     0.330     0.074      0.161   
4      11    -0.055    -0.086     0.294     0.330     0.118      0.025   
5      13     0.003     0.019     0.294     0.312     0.031      0.161   
6      14       NaN       NaN       NaN       NaN       NaN        NaN   
7      16       NaN       NaN       NaN       NaN       NaN        NaN   
8      18    -0.016    -0.041    -0.179    -0.179    -0.056      0.161   
9      23       NaN       NaN       NaN       NaN       NaN        NaN   
10     26     0.016     0.093    -0.015    -0.016     0.031      0.116   
11     27    -0.062    -0.153    -0.197    -0.179    -0.187     -0.384   
12

In [21]:
train_category.describe

<bound method DataFrame.describe of      Id  L0_S1_F25  L0_S1_F27  L0_S1_F29  L0_S1_F31  L0_S2_F33  L0_S2_F35  \
0     4        NaN        NaN        NaN        NaN        NaN        NaN   
1     6        NaN        NaN        NaN        NaN        NaN        NaN   
2     7        NaN        NaN        NaN        NaN        NaN        NaN   
3     9        NaN        NaN        NaN        NaN        NaN        NaN   
4    11        NaN        NaN        NaN        NaN        NaN        NaN   
5    13        NaN        NaN        NaN        NaN        NaN        NaN   
6    14        NaN        NaN        NaN        NaN        NaN        NaN   
7    16        NaN        NaN        NaN        NaN        NaN        NaN   
8    18        NaN        NaN        NaN        NaN        NaN        NaN   
9    23        NaN        NaN        NaN        NaN        NaN        NaN   
10   26        NaN        NaN        NaN        NaN        NaN        NaN   
11   27        NaN        NaN        NaN

In [22]:
from operator import add

counts = sc.parallelize([1,2,3,4]).map(lambda x: x^2).reduce(add)
print type(counts)
print counts
# for (word, count) in output:
#         print("%s: %i" % (word, count))

<type 'int'>
10


In [4]:
cat_data =sc.textFile("/home/h/kaggle_fun/bosch/raw_data/train_categorical.csv")

In [5]:
cat_data.count()

1183748

In [18]:
spark = SparkSession.builder \
    .master("spark://h:7077") \
    .appName("Word Count") \
    .getOrCreate()

In [25]:
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"
train_cat = "train_categorical.csv"
train_numeric = "train_numeric.csv"
train_date= "train_date.csv"
train_cat_df = spark.read.csv(file_dir + train_cat, header=True, inferSchema=True, nanValue="NaN")
train_numerer_df = spark.read.csv(file_dir + train_numeric, header=True, inferSchema=True, nanValue="NaN")
train_date_df = spark.read.csv(file_dir + train_date, header=True, inferSchema=True, nanValue="NaN")

In [26]:
train_cat_df.registerTempTable("tmp_cate")

In [27]:
res = spark.sql("select count(*) from tmp_cate")

In [28]:
print res.collect()

[Row(count(1)=1183747)]


In [30]:
print train_cat_df.printSchema

<bound method DataFrame.printSchema of DataFrame[Id: int, L0_S1_F25: string, L0_S1_F27: string, L0_S1_F29: string, L0_S1_F31: string, L0_S2_F33: string, L0_S2_F35: string, L0_S2_F37: string, L0_S2_F39: string, L0_S2_F41: string, L0_S2_F43: string, L0_S2_F45: string, L0_S2_F47: string, L0_S2_F49: string, L0_S2_F51: string, L0_S2_F53: string, L0_S2_F55: string, L0_S2_F57: string, L0_S2_F59: string, L0_S2_F61: string, L0_S2_F63: string, L0_S2_F65: string, L0_S2_F67: string, L0_S3_F69: string, L0_S3_F71: string, L0_S3_F73: string, L0_S3_F75: string, L0_S3_F77: string, L0_S3_F79: string, L0_S3_F81: string, L0_S3_F83: string, L0_S3_F85: string, L0_S3_F87: string, L0_S3_F89: string, L0_S3_F91: string, L0_S3_F93: string, L0_S3_F95: string, L0_S3_F97: string, L0_S3_F99: string, L0_S3_F101: string, L0_S3_F103: string, L0_S4_F105: string, L0_S4_F107: string, L0_S4_F108: string, L0_S4_F110: string, L0_S4_F112: string, L0_S4_F113: string, L0_S6_F119: string, L0_S6_F121: string, L0_S6_F123: string, 